In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# conda install -c conda-forge imbalanced-learn

In [2]:
# conda update -n base -c defaults conda

In [3]:
# pip install --upgrade imbalanced-learn

In [5]:
pip install SMOTEENN

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement SMOTEENN (from versions: none)
ERROR: No matching distribution found for SMOTEENN


In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from imblearn.metrics import classification_report_imbalanced
from imblearn.combine import SMOTEENN



In [7]:
credit_df = pd.read_csv(r'C:\Users\Columbia bootcamp\Credit_Risk_Analysis\Starter_Code\LoanStats_2019Q1.csv')
credit_df

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
id                                               member_id loan_amnt funded_amnt funded_amnt_inv term       int_rate installment grade sub_grade emp_title                                emp_length home_ownership annual_inc verification_status issue_d  loan_status pymnt_plan url desc purpose            title                   zip_code addr_state dti   delinq_2yrs earliest_cr_line inq_last_6mths mths_since_last_delinq mths_since_last_record open_acc pub_rec revol_bal revol_util total_acc initial_list_status out_prncp out_prncp_inv total_pymnt total_pymnt_inv total_rec_prncp total_rec_int total_rec_late_fee recoveries collection_recovery_fee last_pymnt_d last_pymnt_amnt next_pymnt_d last_credit_pull_d collections_12_mths_ex_med mths_since_last_major_derog policy_code application_type annual_inc_joint dti_joint verification_status_joint acc_now_delinq tot_coll_amt tot_cur_bal open_acc_6m open_act_il open_il_12m open_il_24m mths_since_rcnt_il total_bal_il il_util open_rv_12m open_rv_24m max_bal_bc all_util total_rev_hi_lim inq_fi total_cu_tl inq_last_12m acc_open_past_24mths avg_cur_bal bc_open_to_buy bc_util chargeoff_within_12_mths delinq_amnt mo_sin_old_il_acct mo_sin_old_rev_tl_op mo_sin_rcnt_rev_tl_op mo_sin_rcnt_tl mort_acc mths_since_recent_bc mths_since_recent_bc_dlq mths_since_recent_inq mths_since_recent_revol_delinq num_accts_ever_120_pd num_actv_bc_tl num_actv_rev_tl num_bc_sats num_bc_tl num_il_tl num_op_rev_tl num_rev_accts num_rev_tl_bal_gt_0 num_sats num_tl_120dpd_2m num_tl_30dpd num_tl_90g_dpd_24m num_tl_op_past_12m pct_tl_nvr_dlq percent_bc_gt_75 pub_rec_bankruptcies tax_liens tot_hi_cred_lim total_bal_ex_mort total_bc_limit total_il_high_credit_limit revol_bal_joint sec_app_earliest_cr_line sec_app_inq_last_6mths sec_app_mort_acc sec_app_open_acc sec_app_revol_util sec_app_open_act_il sec_app_num_rev_accts sec_app_chargeoff_within_12_mths sec_app_collections_12_mths_ex_med sec_app_mths_since_last_major_derog hardship_flag hardship_type hardship_reason hardship_status deferral_term hardship_amount hardship_start_date hardship_end_date payment_plan_start_date hardship_length hardship_dpd hardship_loan_status orig_projected_additional_accrued_interest hardship_payoff_balance_amount hardship_last_payment_amount debt_settlement_flag debt_settlement_flag_date settlement_status settlement_date settlement_amount settlement_percentage                                    settlement_term                              
NaN                                              NaN       20000     20000       20000            60 months  17.19%  499.1       C     C5        Front desk supervisor                    6 years    RENT           47000      Source Verified     Mar-2019 Issued      n          NaN NaN  debt_consolidation Debt consolidation      958xx    CA         14.02 0           Sep-2006         1              50                     NaN                    15       0       10687     19.7%      53        w                   20000.00  20000.00      0.0         0.00            0.00            0.00          0.0                0.0        0.0                     NaN          0.0             Apr-2019     Apr-2019           0                          50                          1           Individual       NaN              NaN       NaN                       0              0            31546       3           2           1           2           10                 20859        97      4           9           5909       42       54300            6      1           3            11                   2103        23647          30      0                        0           150                100                  1                     1              0        5                    NaN                      3                     50                             3                     3              4               8           19        19        13            33           

In [8]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

features = columns + target

# Print the combined list
print(features)

['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'm

In [9]:
# Check if each column exists in the DataFrame
#filtered_columns = [col for col in credit_df.columns if col in columns]
filtered_columns = [col for col in columns if col in credit_df.columns]

# Print the filtered columns
print(filtered_columns)

[]


In [10]:
# Check if any columns are left after filtering
if len(filtered_columns) == 0:
    print("No common columns found between the list and the DataFrame.")
else:
    # Create the training variables by converting the string values into numerical ones using the get_dummies() method
    credit_df = pd.get_dummies(df_selected[filtered_columns])

    # Create the target variable
    target_variable = df_selected[target].values.flatten()

    # Check the balance of the target variable
    target_balance = pd.Series(target_variable).value_counts()
    imbalance_ratio = target_balance[1] / target_balance[0]

    # Print the balance information
    print("Target Variable Balance:")
    print(target_balance)
    print("Imbalance Ratio: {:.2f}".format(imbalance_ratio))

No common columns found between the list and the DataFrame.


In [11]:
# Filter the columns that exist in the DataFrame
filtered_columns = [col for col in columns if col in credit_df.columns]

# Create the training variables by converting the string values into numerical ones using the get_dummies() method
credit_df = pd.get_dummies(credit_df[filtered_columns])

# Create the target variable
target_variable = credit_df[target].values.flatten()

# Check the balance of the target variable
target_balance = pd.Series(target_variable).value_counts()
imbalance_ratio = target_balance[1] / target_balance[0]

# Print the balance information
print("Target Variable Balance:")
print(target_balance)
print("Imbalance Ratio: {:.2f}".format(imbalance_ratio))



ValueError: No objects to concatenate

In [ ]:
# Load the data
file_path = Path(r'C:\Users\Columbia bootcamp\Credit_Risk_Analysis\Starter_Code\LoanStats_2019Q1.csv')
df = credit_df
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
#issued_mask = df['loan_status'] != 'Issued'
#df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

df_selected = df
# Convert the tar# Convert the target column values to low_risk and high_risk based on their values
df_selected['loan_status'] = df_selected['loan_status'].map({'Fully Paid': 'low_risk', 'Charged Off': 'high_risk'})

x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df_selected.head()

In [ ]:
# Create our features
df_selected = pd.get_dummies(df_selected.drop(target, axis=1))

# Create our target
target_variable = df_selected[target].values.flatten()
target_balance = pd.Series(target_variable).value_counts()
imbalance_ratio = target_balance[1] / target_balance[0]

In [ ]:
df_selected.describe()

In [ ]:
# Check the balance of our target values
df_selected['loan_status'].value_counts()


In [ ]:
# Resample the training data with the RandomOversampler

print("Before Random Oversampling:")
print(df_selected['loan_status'].value_counts())


random_oversampler = RandomOverSampler(random_state=1)

# Resample the training data
X_resampled, y_resampled = random_oversampler.fit_resample(df_selected.drop('loan_status', axis=1), df_selected['loan_status'])


print("After Random Oversampling:")
print(pd.Series(y_resampled).value_counts())


In [ ]:
# Train the Logistic Regression model using the resampled data

training_data = df_selected
training_data = x
X = training_data.drop('loan_status', axis=1)
y = training_data['loan_status']

logreg = LogisticRegression(random_state=1)

logreg.fit(X_resampled, y_resampled)



In [ ]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score:", balanced_accuracy)



In [ ]:
# Display the confusion matrix

confusion_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion_matrix)

In [ ]:
# Print the imbalanced classification report
classification_report_imbalanced = classification_report(y_test, y_pred)
print("Classification Report (Imbalanced):")
print(classification_report_imbalanced)

In [ ]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=1)

X_resampled, y_resampled = smote.fit_resample(X, y)


In [ ]:
# Train the Logistic Regression model using the resampled data

logreg = LogisticRegression(random_state=1)
logreg.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

print("Balanced Accuracy Score:", balanced_accuracy)

In [ ]:
# Display the confusion matrix
y_pred = logreg.predict(X_test)

# Calculate the confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(confusion_matrix)

In [ ]:
# Print the imbalanced classification report
print("Imbalanced Classification Report:")
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids


In [ ]:
cluster_centroids = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cluster_centroids.fit_resample(X, y)
print("After ClusterCentroids Resampling:")
print(pd.Series(y_resampled).value_counts())


In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=1)
logreg = LogisticRegression(random_state=1)


In [ ]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score:", balanced_acc)


In [ ]:
# Display the confusion matrix
y_pred = logreg.predict(X_test)
confusion_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(confusion_matrix)

In [ ]:
# Print the imbalanced classification report

print("Imbalanced Classification Report:")
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Resample the training data with SMOTEENN
smoteenn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)


In [ ]:
# Train the Logistic Regression model using the resampled data
logreg = LogisticRegression(random_state=1)
logreg.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score:", balanced_accuracy)

In [ ]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Print the imbalanced classification report
y_pred = logreg.predict(X_test)

classification_report = classification_report_imbalanced(y_test, y_pred)

print("Imbalanced Classification Report:")
print(classification_report)
